This notebook works through a process to build a collection of model descriptor items in ScienceBase from a source spreadsheet and then get that information back out of ScienceBase as a spreadsheet. Once the items are in ScienceBase, you could also use the built in output to CSV for a ScienceBase catalog search result, but doing that in code lets you fully control what you want in the spreadsheet.

There is also another feature in ScienceBase to attach a spreadsheet to an item and then provide a configuration snippet that is used to generate child items from the rows in the spreadsheet. I just didn't immediately turn up documentation on how to do that, and I've forgotten the specifics.

Personally, I would use the spreadsheet-to-sciencebase method once and then do everything else in ScienceBase from that point on. Otherwise, you would need a process to first lookup the existing item in ScienceBase and update with whatever you changed in the spreadsheet. I did not build that kind of "upsert" method into this code.

In [1]:
import pandas as pd
from sciencebasepy import SbSession
import requests
import json
from IPython.display import display

# Functions
I put all of the codes needed to build this out into individual functions that do the work. My functions here are lazy in that they make some assumptions that should properly be checked with some error trapping. If I had more time, I'd clean things up a bit. If you follow the instructions, they should all run just fine.

* create_model_catalog - Creates the container item either in the logged in user's My Items container or into another parent of your choosing. Defaults to a particular title and will delete any items with the same title (and child items) if it exists in the specified location.
* load_models_spreadsheet - Loads the models Excel file into a Pandas dataframe, defaults to the last known file name in the local immediate path
* sb_party_to_contact - Takes an email address for a point of contact on the model, searches for the contact in the ScienceBase Directory, and builds a contact document in the format needed by ScienceBase Catalog
* sb_web_link - Takes a URL and optional title and returns a webLink document in the format needed by ScienceBase Catalog
* build_model_documents - Processes the dataframe from the spreadsheet to turn each record into a ScienceBase Item that documents a model
* model_catalog_list_out - Gets all model descriptions from the Model Catalog collection in ScienceBase and outputs them in a number of ways; defaults to building an Excel file with no return output

In [2]:
def create_model_catalog(parent_id=None, title="USGS Model Catalog", body=None, delete_if_exists=True):
    if parent_id is None:
        parent_id = sb.get_my_items_id()
    
    if delete_if_exists:
        existing_items = json.loads(sb.get(
            f"https://www.sciencebase.gov/catalog/items?format=json&parentId={parent_id}&lq=title:{title}"
        ))
        
        if existing_items["total"] > 0:
            for item in existing_items["items"]:
                if item["hasChildren"]:
                    sb.delete_items(sb.get_child_ids(item["id"]))
                sb.delete_item(item)
                
    model_catalog_item = {
        'title': title,
        'parentId': parent_id
    }
    
    if body is not None:
        model_catalog_item["body"] = body

    return sb.create_item(model_catalog_item)


def load_models_spreadsheet(file_path="USGS_models_named_models.xlsx"):
    output_link_columns = ["Output","Output.1","Output.2","Output.3","Output.4"]
    
    usgs_models = pd.read_excel(file_path)

    # Replace NaN with None (makes it simpler to evaluate values)
    usgs_models = usgs_models.replace({pd.np.nan: None})

    # Put all of the output links into a list (makes it easier to process these later)
    usgs_models["output_links"] = usgs_models[output_link_columns].values.tolist()
    usgs_models = usgs_models.drop(columns=output_link_columns)
    
    # Drop any unnamed columns (blanks in the Excel file)
    usgs_models = usgs_models.drop(columns=[i for i in list(usgs_models.columns) if i.find("Unnamed") != -1])
    
    return usgs_models


def sb_party_to_contact(search_term):
    search_result = requests.get(
        f"https://www.sciencebase.gov/directory/people?q={search_term}&format=json&dataset=all&max=10"
    ).json()
    
    if search_result["total"] == 1:
        person_record = search_result["people"][0]

        sb_contact = {
            "name": person_record["displayName"],
            "type": "Contact",
            "oldPartyId": person_record["id"],
            "contactType": person_record["type"],
            "onlineResource": f"https://my.usgs.gov/catalog/Global/catalogParty/show/{person_record['id']}",
            "email": person_record["email"],
            "active": person_record["active"],
            "jobTitle": person_record["extensions"]["personExtension"]["jobTitle"],
            "firstName": person_record["extensions"]["personExtension"]["firstName"],
            "lastName": person_record["extensions"]["personExtension"]["lastName"]
        }

        if "orcId" in person_record.keys():
            sb_contact["orcId"] = person_record["orcId"]

    else:
        sb_contact = {
            "name": search_term,
            "type": "Contact",
            "email": search_term
        }
        
    return sb_contact
    
    
def sb_web_link(url, title="Model Reference Link"):
    return {
                "type": "webLink",
                "typeLabel": "Web Link",
                "uri": url,
                "rel": "related",
                "title": title,
                "hidden": False
            }


def build_model_documents(df_models=None):
    if df_models is None:
        df_models = load_models_spreadsheet()
    
    model_documents = list()

    for index, record in df_models.iterrows():
        new_model_item = {
            "parentId": model_catalog["id"],
            "title": record["Model Name"],
            "webLinks": list()
        }

        # Here we take the contact email addresses and use the sb_party_to_contact function to look them up and make proper contacts for ScienceBase
        record_contacts = record["Contact(s)"].split(";")
        if len(record_contacts) > 0:
            new_model_item["contacts"] = [sb_party_to_contact(contact) for contact in record_contacts]
            
        # Here we split the sometimes lists of model reference links and add them to web links
        for link in record["Link"].split(";"):
            new_model_item["webLinks"].append(sb_web_link(link))

        # Here we filter down to just output link values not already processed as a model reference and containing an actual value
        for link in [l for l in record["output_links"] if l is not None and len(l.strip()) > 0 and not l in [i["uri"] for i in new_model_item["webLinks"]]]:
            new_model_item["webLinks"].append(sb_web_link(link, "Model Output Data"))

        model_documents.append(new_model_item)
        
    return model_documents


def model_catalog_list_out(
    model_catalog_id, 
    include_contact=True, 
    include_ref_link=True,
    include_sb_link=True,
    return_data=None, 
    write_to_excel=True, 
    file_name="usgs_model_catalog.xlsx"
):
    simple_model_list = list()
    column_list = ["Model Name"]
    if include_contact:
        column_list.append("Contact")

    if include_ref_link:
        column_list.append("Model Reference Link")

    if include_sb_link:
        column_list.append("ScienceBase Link")

    items = sb.find_items({"parentId": model_catalog_id, "fields": "title,webLinks,contacts"})
    while items and 'items' in items:
        for item in items['items']:
            simple_item = {
                "Model Name": item['title']
            }

            if include_contact:
                simple_item["Contact"] = next((c["name"] for c in item["contacts"]), None)
                
            if include_ref_link:
                simple_item["Model Reference Link"] = next((l["uri"] for l in item["webLinks"] if l["title"] == "Model Reference Link"), None)
            
            if include_sb_link:
                simple_item["ScienceBase Link"] = item["link"]["url"]

            simple_model_list.append(simple_item)

        items = sb.next(items)
        
    df_model_list = pd.DataFrame(simple_model_list)

    if write_to_excel:
        df_model_list.to_excel(file_name, index=False, columns=column_list)
    
    if return_data == "dataframe":
        return df_model_list
    
    if return_data == "dict":
        return simple_model_list

# Connect to ScienceBase
You need to establish an authenticated session with ScienceBase in order to write any items to the system. Insert your email address here, and when you execute the cell, you will be prompted for your Active Directory password.

In [3]:
sb = SbSession().loginc("sbristol@usgs.gov")

········


# Create Model Catalog Container
You will need to put the top level "USGS Model Catalog" item somewhere to serve as the container/collection for the model descriptive items. I would personally put that up at the very root of ScienceBase, eventually, but for now that can go anywhere that you want to house it in the near term. It just needs to be somewhere that you or someone else can open up for public access.

Here, I am getting my own "My Items" ID using a function in sciencebasepy. You can replace parent_id_for_catalog with whatever ScienceBase Item ID (UUID) value you want to use or put it in your My Items space. We just want to be careful not to proliferate too many of these and clean up after ourselves.

We then fire the create_model_catalog function with default settings for title and an option to delete any items with that same title in the parent container if they exist. We then look at the full item document that we just created. The ID for this item now becomes the parent ID that we'll use in creating the actual individual model descriptor items from the spreadsheet.

In [4]:
model_catalog = create_model_catalog()
model_catalog

{'link': {'rel': 'self',
  'url': 'https://www.sciencebase.gov/catalog/item/5e8de96182cee42d134687cc'},
 'relatedItems': {'link': {'url': 'https://www.sciencebase.gov/catalog/itemLinks?itemId=5e8de96182cee42d134687cc',
   'rel': 'related'}},
 'id': '5e8de96182cee42d134687cc',
 'title': 'USGS Model Catalog',
 'provenance': {'dateCreated': '2020-04-08T15:10:25Z',
  'lastUpdated': '2020-04-08T15:10:25Z',
  'lastUpdatedBy': 'sbristol@usgs.gov',
  'createdBy': 'sbristol@usgs.gov'},
 'hasChildren': False,
 'parentId': '4f4f863be4b0c2aeb78a9e3f',
 'permissions': {'read': {'acl': [],
   'inherited': True,
   'inheritsFromId': '4f4f863be4b0c2aeb78a9e3f'},
  'write': {'acl': ['USER:sbristol@usgs.gov'],
   'inherited': True,
   'inheritsFromId': '4f4f863be4b0c2aeb78a9e3f'}},
 'distributionLinks': [],
 'locked': False}

# Load the Excel file for processing
There are lots of ways of working with spreadsheets of different kinds, but Pandas is pretty simple and convenient. Here we use the function to read the latest snapshot of the Excel file that I put in the code repo into a Pandas dataframe and show what it looks like.

In [5]:
usgs_models = load_models_spreadsheet()
usgs_models.head()

,Model Name,Link,Contact(s),output_links
0,1DTempPro,https://water.usgs.gov/ogw/bgas/1dtemppro/,edswain@usgs.gov,"[https://doi.org/10.5066/P9Q8JGAO, , None, No..."
1,BBS,https://www.mbr-pwrc.usgs.gov/bbs/,sbeliew@usgs.gov,"[https://doi.org/10.5066/F7JS9NHH, , None, No..."
2,BEWARE,https://catalog.data.gov/dataset/beware-databa...,aallwardt@usgs.gov,"[https://doi.org/10.5066/F7T43S20, , None, No..."
3,BISECT,https://pubs.er.usgs.gov/publication/sir20195045,edswain@usgs.gov,"[ , None, None, None, None]"
4,California Basin Characterization Model,https://ca.water.usgs.gov/projects/reg_hydro/b...,lflint@usgs.gov,"[None, None, None, None, None]"


# Processing work
Looking at the data, we need to do a couple of things:

* Split contacts on semicolons for cases where there is more than one email address
* Grab all the output links from separate columns so we can add them as links

We'll check to make sure that any output links aren't already the same as the info link so we don't duplicate those unnecessarily in ScienceBase.

In this process, we need to make some choices about how we are going to deal with laying the information out.

* With the email addresses, we have a good source to use in finding the full person record for contacts we want to add. We can't necessarily assume these are "authors" at this point, so we'll put them in as simple "Point of Contact" type contacts for this context.
* We could do some sleuthing to really classify the links in different ways that could be really useful for the eventual catalog. For now, we can refer to the "Link" links with a title like "Model Reference Link" to basically put those links into a common context here. We'll title the others as "Model Output" for the time being.

Some really interesting things could start to spool out from this just by gathering the links together like this. Some of the links represent machine-readable end points where code could be written to gather structured metadata from those sources and use it to build out a more complete picture of the models. We can explore what that might look like down the road.

# Build the model description documents
The main task here is to take the information from the spreadsheet (title, contacts, web links) about the models and create initial descriptive items to put into ScienceBase. These "description documents" can then be fleshed out further within ScienceBase using both the edit interface and processes we can build later to use the item information as a start and then go out and gather new and fresh information from the web about the models.

One issue here is that the contact matching should really be handled on the ScienceBase API end. I added in a step to lookup actual contact information and identifiers for the email addresses from the ScienceBase Directory. This serves to connect the dots between these new model descriptions we are inserting and named identities associated with them. The most visible effect of this is that you'll be able to hover over the contacts inserted into ScienceBase, link through to the contact pages, and see these new items associated with that contact. Repository management wise, this helps to further flesh out the official record of scientific products associated with an individual researcher.

We shouldn't have to do this operation on the client side in an operation like this. Rather, ScienceBase should be able to connect the dots for us simply by us passing in a known identifier like a usgs.gov email address. We wouldn't want ScienceBase to automatically connect to a Directory identity for every contact like this, but if it's a usgs.gov email (and possibly other DOI emails) AND the identity is current in the Directory, then it would be reasonable to have that connection made automatically for us.

In [6]:
model_docs = build_model_documents(usgs_models)

We can take a look now at what the barebones ScienceBase Items will look like for the model descriptions. The parentId points to the new container we created to house and instantiate the USGS Model Catalog. We can see that we've fleshed out the web links, putting them into some logical categories with titles, and built out contacts as full identities linked to our ScienceBase Directory.

The web links here are going to be likely the most important part of the eventual description information model for these items. They are going to do things like drive processes to continually monitor the web for new and refreshed information about the models. Classifying the links to clarify what function they serve will be an important part of driving this process.

### Slight rant...
Unfortunately, the ScienceBase vocabularies for [web link types](https://www.sciencebase.gov/vocab/4f4e475de4b07f02db47debe/terms) and [contact types](https://www.sciencebase.gov/vocab/4f4e475ce4b07f02db47de84/terms) are really a mess right now and virtually unusable for any real purposes. In both groups, some types are incredibly detailed while others are totally vague. I could have picked link types other than the generic "webLink" to try and classify these, but it would have been meaningless without working on the semantics for the vocabulary. As soon as any vocabulary gets this large and messy, it is incredibly important that the owners of the vocabulary provide actual working definitions for the terms through descriptions and/or relationships to other vocabularies. What we have now means that each collection has to essentially bring its own contextual meaning to its particular use of the vocabulary. Someone really needs to work on ScienceBase Catalog semantics.

In [7]:
print(len(model_docs))
display(model_docs[18])

36


{'parentId': '5e8de96182cee42d134687cc',
 'title': 'MODFLOW-LGR',
 'webLinks': [{'type': 'webLink',
   'typeLabel': 'Web Link',
   'uri': 'https://water.usgs.gov/ogw/modflow-lgr/',
   'rel': 'related',
   'title': 'Model Reference Link',
   'hidden': False},
  {'type': 'webLink',
   'typeLabel': 'Web Link',
   'uri': 'https://pubs.usgs.gov/tm/6a44/',
   'rel': 'related',
   'title': 'Model Output Data',
   'hidden': False},
  {'type': 'webLink',
   'typeLabel': 'Web Link',
   'uri': 'https://doi.org/10.3133/tm6A44',
   'rel': 'related',
   'title': 'Model Output Data',
   'hidden': False}],
 'contacts': [{'name': 'smehl@csuchico.edu',
   'type': 'Contact',
   'email': 'smehl@csuchico.edu'},
  {'name': 'GS-CAWSC MODFLOW OWHM',
   'type': 'Contact',
   'oldPartyId': 66925,
   'contactType': 'person',
   'onlineResource': 'https://my.usgs.gov/catalog/Global/catalogParty/show/66925',
   'email': 'modflow_owhm@usgs.gov',
   'active': False,
   'jobTitle': None,
   'firstName': 'GS-CAWSC',
 

# Create items in catalog
Now that we have our item documents built, we can go ahead and commit them to ScienceBase. We bring the committed items back into their own variable so we can see what they look like once created.

In [8]:
model_descriptor_items = sb.create_items(model_docs)

In [9]:
print(len(model_descriptor_items))
display(model_descriptor_items[18])

36


{'link': {'rel': 'self',
  'url': 'https://www.sciencebase.gov/catalog/item/5e8de97782cee42d134687f3'},
 'relatedItems': {'link': {'url': 'https://www.sciencebase.gov/catalog/itemLinks?itemId=5e8de97782cee42d134687f3',
   'rel': 'related'}},
 'id': '5e8de97782cee42d134687f3',
 'title': 'MODFLOW-LGR',
 'provenance': {'dateCreated': '2020-04-08T15:10:47Z',
  'lastUpdated': '2020-04-08T15:10:47Z',
  'lastUpdatedBy': 'sbristol@usgs.gov',
  'createdBy': 'sbristol@usgs.gov'},
 'hasChildren': False,
 'parentId': '5e8de96182cee42d134687cc',
 'contacts': [{'name': 'smehl@csuchico.edu',
   'type': 'Contact',
   'email': 'smehl@csuchico.edu'},
  {'name': 'GS-CAWSC MODFLOW OWHM',
   'oldPartyId': 66925,
   'type': 'Contact',
   'contactType': 'person',
   'onlineResource': 'https://my.usgs.gov/catalog/Global/catalogParty/show/66925',
   'email': 'modflow_owhm@usgs.gov',
   'firstName': 'GS-CAWSC',
   'lastName': 'MODFLOW OWHM'}],
 'webLinks': [{'type': 'webLink',
   'typeLabel': 'Web Link',
   'ur

# Items to spreadsheet
Now that we have our simple prototype USGS Model Catalog created and populated it with our model descriptions, we can continue fleshing things out with additional details such as other web links, relationships to other ScienceBase Items, and other details we want to add. If we want to get our catalog back out as a simple spreadsheet, we can use the built in functions for ScienceBase Catalog search to dump out a CSV, which gives us a canned set of fields to work with, or we can do it with code to customize what we get back.

I wrapped all of the item retrieval and output steps into a single function that only requires the model catalog parent ID to be passed in. (Note that this will essentially return something for any parent ID that you would send.) You can set the return_data parameter to return the data as either a list of dictionaries or as a Pandas dataframe. You can also set a parameter to either write out the Excel file or not, and you can specify which specific output fields you want to include.

I made a number of choices about how to handle this part of the process with an eye toward providing a very simple output for our immediate purpose of simply sharing that we are making some degree of progress on the effort. By default, the resulting Excel file will have the Model Name, the first Model Reference Link, the first Contact (full name drawn from ScienceBase Directory where available), and will include a link to the ScienceBase Item created. We would only want to include the ScienceBase Link if we actually make the catalog public so those will be meaningful to anyone we share the spreadsheet with.

In the following code block, I run the basic function and return it as a dataframe to show the basic structure created. Note that the column ordering is handled differently for the Excel output if you check out the generated file.

Note that there can be a time lag in ScienceBase fully creating items and populating its search index. If you simply run all codeblocks in this notebook, you might get an Excel output that does not include every item. Wait for a few seconds and try running the model_catalog_list_out function again, and you should get all records dumped to the spreadsheet file.

In [11]:
model_catalog_list_out(model_catalog["id"], return_data="dataframe").head()

,Contact,Model Name,Model Reference Link,ScienceBase Link
0,Lorraine E Flint,California Basin Characterization Model,https://ca.water.usgs.gov/projects/reg_hydro/b...,https://www.sciencebase.gov/catalog/item/5e8de...
1,John C Warner,COAWST,https://www.usgs.gov/center-news/coupled-ocean...,https://www.sciencebase.gov/catalog/item/5e8de...
2,Richard Niswonger,GSFLOW,https://www.usgs.gov/software/coupled-ground-w...,https://www.sciencebase.gov/catalog/item/5e8de...
3,Thomas R Sando,PROSPER,https://www.usgs.gov/centers/wy-mt-water/scien...,https://www.sciencebase.gov/catalog/item/5e8de...
4,Shannon Beliew,BBS,https://www.mbr-pwrc.usgs.gov/bbs/,https://www.sciencebase.gov/catalog/item/5e8de...
